In [2]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from ieeg.auth import Session
from scipy import signal as sig
import os
from os.path import join as ospj
from os.path import exists as ospe
import pathlib
from tqdm import tqdm

In [3]:
import sys
sys.path.append('/users/wojemann/iEEG_processing')
from pioneer import Pioneer

In [7]:
with open('config.json','r') as f:
    CONFIG = json.load(f)
usr = CONFIG["paths"]["iEEG_USR"]
passpath = CONFIG["paths"]["iEEG_PWD"]
raw_datapath = CONFIG["paths"]["RAW_DATA"]
figpath = CONFIG["paths"]["FIGURES"]
patient_table = pd.DataFrame(CONFIG["patients"]).sort_values('ptID')
rid_hup = pd.read_csv(ospj(raw_datapath,'rid_hup.csv'))
pt_list = patient_table.ptID.to_numpy()

In [8]:
pt_list

array(['HUP170', 'HUP172', 'HUP180', 'HUP181', 'HUP186', 'HUP188',
       'HUP193', 'HUP194', 'HUP197', 'HUP201', 'HUP202', 'HUP205',
       'HUP207', 'HUP210', 'HUP211', 'HUP213', 'HUP214', 'HUP216',
       'HUP218', 'HUP224', 'HUP225', 'HUP230', 'HUP235', 'HUP238',
       'HUP246', 'HUP247', 'HUP249', 'HUP250', 'HUP253', 'HUP257',
       'HUP263', 'HUP266', 'HUP267', 'HUP273'], dtype=object)

In [9]:
for i,pt in patient_table.iloc[-2:-1,:].iterrows():
    for ieeg_pt in pt.ieeg_ids:
        try:
            save_path = ospj(raw_datapath,pt.ptID)
            print(ieeg_pt,save_path)
            if not ospe(save_path):
                os.makedirs(save_path)
            wagon = Pioneer(usr,passpath,ieeg_pt)
            wagon.pull_annotations()
            wagon.filter_seizure_annotations()
            wagon.seizure_annotations.to_csv(ospj(save_path,f'seizure_annotations_{ieeg_pt}.csv'))
        except:
            continue

HUP267_phaseII /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP267
got 100 annotations on call # 1 covering 2873046 usec to 52508947756 usec
got 100 annotations on call # 2 covering 52525947756 usec to 54168947756 usec
got 100 annotations on call # 3 covering 54179947756 usec to 56855947756 usec
got 100 annotations on call # 4 covering 56887947756 usec to 58277656740 usec
got 100 annotations on call # 5 covering 58290656740 usec to 59254954592 usec
got 100 annotations on call # 6 covering 59255961427 usec to 60149000490 usec
got 100 annotations on call # 7 covering 60150004396 usec to 61172704592 usec
got 100 annotations on call # 8 covering 61173656740 usec to 61873656740 usec
got 100 annotations on call # 9 covering 61898101076 usec to 62547883302 usec
got 100 annotations on call # 10 covering 62550656740 usec to 63281775881 usec
got 100 annotations on call # 11 covering 63288656740 usec to 63988280763 usec
got 100 annotations on call # 12 covering 63992824709 usec to 6542

#### Goal of next section is to make a csv containing all of the seizure times for each of these patients. The csv should contain HUPID, EEC, UEO, flag for stim or not, 